In [ ]:
# 2. Haar Cascade Object detection for Eye and Face in Python using Open CV
import cv2
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
def detect_faces_and_eyes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=5)
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = image[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(
            roi_gray, scaleFactor=1.1, minNeighbors=3)
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
    return image
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    result_frame = detect_faces_and_eyes(frame)
    cv2.imshow('Result Frame', result_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


In [ ]:
# 1. Teachable Machine Brain.JS In Browser Object Recognition through
import socketio
import base64
import numpy as np
import cv2
sio = socketio.Client()
def preprocess_image(image_data):
    decoded_image_data = base64.b64decode(image_data)
    image_array = np.frombuffer(decoded_image_data, dtype=np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized_image = cv2.resize(gray, (224, 224), interpolation=cv2.INTER_AREA)
    normalized_image = resized_image.astype(np.float32) 
    reshaped_image = np.reshape(normalized_image, (1, 224, 224, 1))
    return reshaped_image
@sio.on('predict')
def on_predict(data):
    preprocessed_image = preprocess_image(data['image'])
    predictions = model.predict(preprocessed_image)
    prediction_dict = {'predictions': predictions.tolist()}
    sio.emit('prediction_result', prediction_dict)
model = socketio.Client().socket("/model")
model.connect()
model.emit('load')
sio.connect('http://localhost:8000')
sio.wait()
